# **Basic Feature Engineering in Keras**

**Learning Objectives**
1. Create an input pipeline using `tf.data`
2. Engineer features to create categorical, crossed, and numerical feature columns

## **Introduction**

In this lab, we utilise feature engineering to improve the prediction of housing prices using a Keras sequential model.

Start by importing the necessary libraries for this lab

In [3]:
import os
import tensorflow.keras

import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from keras.utils import plot_model

print("TensorFlow version", tf.__version__)

TensorFlow version 2.4.1
